# Test nombre lignes siretise

Copy paste from Coda to fill the information

## Objective(s)

- Lors de [l’US 7: Test nombre lignes siretise avec nouvelles regles de gestion](https://coda.io/d/CreditAgricole_dCtnoqIftTn/US-07-ETS-version-3_su0VF), nous avons créé une table avec l’ensemble des possibilités de tests, trié par ordre de préférence. 
  - Afin de séparer les doublons, il suffit de récupérer le rank minimum par index. Celui ci va nous donner le meilleur des probables. 
  - Il est bien sur possible d’avoir encore des doublons, dans ces cas la, il faut aller plus loin dans la rédaction des tests
  - L’objectif de cette US est de récupérer le rank minimum de la table ets_inpi_insee_cases puis de faire une analyse brève des index récupérés. 

## Metadata 

- - Metadata parameters are available here: [Ressources_suDYJ#_luZqd](http://Ressources_suDYJ#_luZqd)

  - Task type:

    - Jupyter Notebook

  - Users: :

    - [Thomas Pernet](mailto:t.pernetcoudrier@gmail.com)

  - Watchers:

    - [Thomas Pernet](mailto:t.pernetcoudrier@gmail.com)

  - Estimated Log points:

    - One being a simple task, 15 a very difficult one
        -  8

  - Task tag

    -  \#sql-query,#matching,#similarite,#regle-de-gestion

  - Toggl Tag

    - \#data-analysis
  
## Input Cloud Storage [AWS/GCP]

If link from the internet, save it to the cloud first

### Tables [AWS/BigQuery]

- - Batch 1:

  - Select Provider: Athena

    - Select table(s): ets_inpi_insee_cases

    - Select only tables created from the same notebook, else copy/paste selection to add new input tables

      - If table(s) does not exist, add them: 

        Add New Table

      - Information:

      - Region: 

        - NameEurope (Paris)
          - Code: eu-west-3

        - Database: inpi

        - Notebook construction file: [07_pourcentage_siretisation_v3](https://github.com/thomaspernet/InseeInpi_matching/blob/master/Notebooks_matching/Data_preprocessed/programme_matching/02_siretisation/07_pourcentage_siretisation_v3.md)
    
## Destination Output/Delivery

- AWS

    -  Athena: 

      -  Region: Europe (Paris)
        - Database: inpi
        - Tables (Add name new table): ets_inpi_insee_cases_rank
        - List new tables
        - ets_inpi_insee_cases_rank 

## Things to know (Steps, Attention points or new flow of information)

### Sources of information  (meeting notes, Documentation, Query, URL)


## Connexion serveur

In [1]:
from awsPy.aws_authorization import aws_connector
from awsPy.aws_athena import service_athena
from awsPy.aws_s3 import service_s3
from pathlib import Path
import pandas as pd
import numpy as np
import os, shutil
bucket = 'calfdata'
path = os.getcwd()
parent_path = str(Path(path).parent)
path_cred = r"{}/credential_AWS.json".format(parent_path)

region = 'eu-west-3'
bucket = 'calfdata'

In [2]:
con = aws_connector.aws_instantiate(credential = path_cred,
                                       region = region)
client= con.client_boto()
s3 = service_s3.connect_S3(client = client,
                      bucket = bucket, verbose = False) 
#athena = service_athena.connect_athena(client = client,
#                      bucket = bucket) 
s3_output = 'INPI/sql_output'

In [3]:
import seaborn as sns

cm = sns.light_palette("green", as_cmap=True)
pd.set_option('display.max_columns', None)

# Creation tables

## Steps

- Recupération du rank minimum par index
- Filtre les lignes correspondant au rank minimum par index

In [ ]:
query = """
CREATE TABLE inpi.ets_inpi_insee_cases_rank
WITH (
  format='PARQUET'
) AS
WITH tb_min_rank AS (
SELECT 
min_rank,
  ets_inpi_insee_cases.index_id, 
  row_id, 
  sequence_id, 
  siren, 
  siret, 
  count_initial_insee, 
  count_inpi_siren_siret, 
  count_inpi_siren_sequence, 
  count_inpi_sequence_siret, 
  count_inpi_sequence_stat_cas_siret, 
  count_inpi_index_id_siret, 
  count_inpi_index_id_stat_cas_siret, 
  count_inpi_index_id_stat_cas, 
  index_id_duplicate, 
  test_sequence_siret, 
  test_index_siret, 
  test_siren_insee_siren_inpi, 
  test_sequence_siret_many_cas, 
  list_numero_voie_matching_inpi, 
  list_numero_voie_matching_insee, 
  intersection_numero_voie, 
  union_numero_voie, 
  test_list_num_voie, 
  datecreationetablissement, 
  date_debut_activite, 
  test_date, 
  etatadministratifetablissement, 
  status_admin, 
  test_status_admin, 
  etablissementsiege, 
  status_ets, 
  test_siege, 
  codecommuneetablissement, 
  code_commune, 
  test_code_commune, 
  codepostaletablissement, 
  code_postal_matching, 
  numerovoieetablissement, 
  numero_voie_matching, 
  test_numero_voie, 
  typevoieetablissement, 
  type_voie_matching, 
  test_type_voie, 
  list_inpi, 
  lenght_list_inpi, 
  list_insee, 
  lenght_list_insee, 
  inpi_except, 
  insee_except, 
  intersection, 
  union_, 
  pct_intersection, 
  len_inpi_except, 
  len_insee_except, 
  status_cas, 
  test_adresse_cas_1_3_4, 
  index_id_dup_has_cas_1_3_4, 
  test_duplicates_is_in_cas_1_3_4, 
  enseigne, 
  enseigne1etablissement, 
  enseigne2etablissement, 
  enseigne3etablissement, 
  test_enseigne 
FROM ets_inpi_insee_cases 
INNER JOIN (
  SELECT index_id, MIN(rank) AS min_rank
FROM ets_inpi_insee_cases
GROUP BY index_id
  ) as tb_min_rank
ON ets_inpi_insee_cases.index_id = tb_min_rank.index_id AND
ets_inpi_insee_cases.rank = tb_min_rank.min_rank
  ) 
  SELECT 
  min_rank,
  tb_min_rank.index_id, 
  count_index,
  row_id, 
  sequence_id, 
  siren, 
  siret, 
  count_initial_insee, 
  count_inpi_siren_siret, 
  count_inpi_siren_sequence, 
  count_inpi_sequence_siret, 
  count_inpi_sequence_stat_cas_siret, 
  count_inpi_index_id_siret, 
  count_inpi_index_id_stat_cas_siret, 
  count_inpi_index_id_stat_cas, 
  index_id_duplicate, 
  test_sequence_siret, 
  test_index_siret, 
  test_siren_insee_siren_inpi, 
  test_sequence_siret_many_cas, 
  list_numero_voie_matching_inpi, 
  list_numero_voie_matching_insee, 
  intersection_numero_voie, 
  union_numero_voie, 
  test_list_num_voie, 
  datecreationetablissement, 
  date_debut_activite, 
  test_date, 
  etatadministratifetablissement, 
  status_admin, 
  test_status_admin, 
  etablissementsiege, 
  status_ets, 
  test_siege, 
  codecommuneetablissement, 
  code_commune, 
  test_code_commune, 
  codepostaletablissement, 
  code_postal_matching, 
  numerovoieetablissement, 
  numero_voie_matching, 
  test_numero_voie, 
  typevoieetablissement, 
  type_voie_matching, 
  test_type_voie, 
  list_inpi, 
  lenght_list_inpi, 
  list_insee, 
  lenght_list_insee, 
  inpi_except, 
  insee_except, 
  intersection, 
  union_, 
  pct_intersection, 
  len_inpi_except, 
  len_insee_except, 
  status_cas, 
  test_adresse_cas_1_3_4, 
  index_id_dup_has_cas_1_3_4, 
  test_duplicates_is_in_cas_1_3_4, 
  enseigne, 
  enseigne1etablissement, 
  enseigne2etablissement, 
  enseigne3etablissement, 
  test_enseigne 
  FROM tb_min_rank
  LEFT JOIN (
    SELECT index_id, COUNT(*) AS count_index
    FROM tb_min_rank
    GROUP BY index_id
    ) as tb_nb_index
    ON tb_min_rank.index_id = tb_nb_index.index_id
"""

output = s3.run_query(
            query=query,
            database='inpi',
            s3_output=s3_output,
  filename = None, ## Add filename to print dataframe
  destination_key = None ### Add destination key if need to copy output
        )

# Analyse

## Count nombre lignes  & index

Le nombre de lignes est de:

In [7]:
query = """
SELECT COUNT(*)
FROM ets_inpi_insee_cases_rank 
"""

output = s3.run_query(
            query=query,
            database='inpi',
            s3_output='INPI/sql_output',
      filename = 'cnt_nb_lignes_rank', ## Add filename to print dataframe
      destination_key = None ### Add destination key if need to copy output
        )
output

Execution ID: 4c35f15f-a463-4a0b-9768-a3d7c3c6fb74


,_col0
0,9446844


Le nombre d'index est de 

In [8]:
query = """
SELECT COUNT(distinct(index_id))
FROM ets_inpi_insee_cases_rank 
"""

output = s3.run_query(
            query=query,
            database='inpi',
            s3_output='INPI/sql_output',
      filename = 'cnt_nb_index_rank', ## Add filename to print dataframe
      destination_key = None ### Add destination key if need to copy output
        )
output

Execution ID: bda44f34-72e1-44ad-8bcb-f0249e6e40b0


,_col0
0,9421163


## Evaluation des doublons

In [18]:
query = """
SELECT count_index, COUNT(*) as ligne_dup
FROM ets_inpi_insee_cases_rank 
GROUP BY count_index 
ORDER BY count_index
"""

nb_ligne = s3.run_query(
            query=query,
            database='inpi',
            s3_output='INPI/sql_output',
      filename = 'cnt_nb_dup_lignes_rank', ## Add filename to print dataframe
      destination_key = None ### Add destination key if need to copy output
)

Execution ID: 131b8604-265a-4f1b-989e-87c6d6a71b79


In [19]:
query = """
SELECT count_index, COUNT(DISTINCT(index_id)) as index_dup
FROM ets_inpi_insee_cases_rank 
GROUP BY count_index 
ORDER BY count_index
"""

nb_index = s3.run_query(
            query=query,
            database='inpi',
            s3_output='INPI/sql_output',
      filename = 'cnt_nb_dup_index_rank', ## Add filename to print dataframe
      destination_key = None ### Add destination key if need to copy output
        )

Execution ID: 7a152a3b-a93c-4aa7-97c7-46c687568ad7


In [26]:
(
pd.concat([    
 pd.concat([
    pd.concat(
    [
        nb_ligne.sum().to_frame().T.rename(index = {0:'total'}), 
        nb_ligne
    ], axis = 0),
    ],axis = 1,keys=["Lignes"]),
    (
 pd.concat([
    pd.concat(
    [
        nb_index.sum().to_frame().T.rename(index = {0:'total'}), 
        nb_index
    ], axis = 0),
    ],axis = 1,keys=["Index"])
)],axis= 1
    )
    .style
    .format("{:,.0f}")
                  .bar(subset= [
                      ('Lignes','ligne_dup'),
                      ('Index','index_dup'),
                      
                  ],
                       color='#d65f5f')
)

## Analyse des ranks

In [31]:
pd.set_option('display.max_colwidth', None)

In [32]:
query = """
SELECT 
count_index, 
  approx_percentile(min_rank, ARRAY[.1, .15, .20, 0.25,0.50,0.75,.80,.85,.86,.87, .88, .89,.90,0.95, 0.99]) as pct_min_rank
FROM 
  ets_inpi_insee_cases_rank
GROUP BY count_index  
ORDER BY count_index
"""
s3.run_query(
            query=query,
            database='inpi',
            s3_output='INPI/sql_output',
      filename = 'distribution_rank', ## Add filename to print dataframe
      destination_key = None ### Add destination key if need to copy output
        )

Execution ID: fe9fcb5a-b46b-4c36-bca2-3288cfec993d


,count_index,pct_min_rank
0,1,"[2, 2, 2, 2, 8, 11, 29, 62, 62, 65, 71, 83, 83, 92, 146]"
1,2,"[2, 2, 2, 8, 8, 11, 39, 83, 83, 89, 89, 91, 92, 127, 146]"
2,3,"[2, 3, 11, 11, 83, 119, 125, 146, 146, 146, 146, 146, 146, 146, 147]"
3,4,"[2, 11, 11, 12, 91, 136, 146, 146, 146, 146, 146, 146, 146, 146, 152]"
4,5,"[11, 12, 12, 12, 92, 120, 137, 146, 146, 146, 146, 146, 146, 146, 147]"
5,6,"[2, 11, 11, 38, 83, 146, 146, 146, 146, 146, 146, 146, 146, 146, 146]"
6,7,"[3, 3, 3, 11, 92, 119, 119, 146, 146, 146, 146, 146, 146, 146, 146]"
7,8,"[2, 2, 30, 30, 65, 92, 119, 146, 146, 146, 146, 146, 146, 146, 146]"
8,9,"[2, 2, 2, 92, 146, 146, 146, 146, 146, 146, 152, 152, 152, 152, 152]"
9,10,"[2, 2, 2, 2, 11, 38, 56, 137, 137, 137, 137, 137, 137, 137, 137]"


In [34]:
query ="""
SELECT *
FROM regles_tests 
WHERE rank = 146
"""
s3.run_query(
            query=query,
            database='inpi',
            s3_output='INPI/sql_output',
      filename = 'rules', ## Add filename to print dataframe
      destination_key = None ### Add destination key if need to copy output
        )

Execution ID: 52bad094-00a9-4b9d-a25b-c948c31c1445


,status_cas,test_list_num_voie,test_siege,test_enseigne,rank
0,CAS_7,NaN,True,NaN,146


## Analyse des cas pour index unique

In [50]:
query = """
SELECT status_cas, count(*) as nb_unique
FROM ets_inpi_insee_cases_rank 
WHERE count_index = 1
GROUP BY status_cas
ORDER BY status_cas
"""

tb = s3.run_query(
            query=query,
            database='inpi',
            s3_output='INPI/sql_output',
      filename = 'nb_cas_index_unique_rank', ## Add filename to print dataframe
      destination_key = None ### Add destination key if need to copy output
        )
(
    tb.
assign(
cumsum = lambda x: x['nb_unique'].cumsum(),
    pct_total = lambda x: x['nb_unique']/x['nb_unique'].sum(),
    pct_total_cum = lambda x: x['pct_total'].cumsum()
)
    .style
    .format("{0:,.2%}", subset=["pct_total", 'pct_total_cum'])
    .bar(subset= ['cumsum'],
                       color='#d65f5f')
    .bar(subset= ['nb_unique'],
                       color='#228B22')
)

Execution ID: 3a6dcd66-866c-40e9-91da-032b23cd05c2


,status_cas,nb_unique,cumsum,pct_total,pct_total_cum
0,CAS_1,7493623,7493623,79.74%,79.74%
1,CAS_3,326180,7819803,3.47%,83.21%
2,CAS_4,457789,8277592,4.87%,88.08%
3,CAS_5,769199,9046791,8.18%,96.26%
4,CAS_6,141300,9188091,1.50%,97.77%
5,CAS_7,209912,9398003,2.23%,100.00%


# Generation report

In [ ]:
import os, time, shutil, urllib, ipykernel, json
from pathlib import Path
from notebook import notebookapp

In [ ]:
def create_report(extension = "html"):
    """
    Create a report from the current notebook and save it in the 
    Report folder (Parent-> child directory)
    
    1. Exctract the current notbook name
    2. Convert the Notebook 
    3. Move the newly created report
    
    Args:
    extension: string. Can be "html", "pdf", "md"
    
    
    """
    
    ### Get notebook name
    connection_file = os.path.basename(ipykernel.get_connection_file())
    kernel_id = connection_file.split('-', 1)[0].split('.')[0]

    for srv in notebookapp.list_running_servers():
        try:
            if srv['token']=='' and not srv['password']:  
                req = urllib.request.urlopen(srv['url']+'api/sessions')
            else:
                req = urllib.request.urlopen(srv['url']+ \
                                             'api/sessions?token=' + \
                                             srv['token'])
            sessions = json.load(req)
            notebookname = sessions[0]['name']
        except:
            pass  
    
    sep = '.'
    path = os.getcwd()
    #parent_path = str(Path(path).parent)
    
    ### Path report
    #path_report = "{}/Reports".format(parent_path)
    #path_report = "{}/Reports".format(path)
    
    ### Path destination
    name_no_extension = notebookname.split(sep, 1)[0]
    source_to_move = name_no_extension +'.{}'.format(extension)
    dest = os.path.join(path,'Reports', source_to_move)
    
    ### Generate notebook
    os.system('jupyter nbconvert --no-input --to {} {}'.format(
    extension,notebookname))
    
    ### Move notebook to report folder
    #time.sleep(5)
    shutil.move(source_to_move, dest)
    print("Report Available at this adress:\n {}".format(dest))

In [ ]:
create_report(extension = "html")